# Homework 7
#### Cody Costa, David Loi

### Problem 1:

### Import necessary module(s)

In [33]:
import numpy as np

### Function to generate random symmetric matrix:

In [34]:
''' function to generate 2x2 symmetric matrix whose
    elements are constrained to be between 0 and 10 '''

size: int = 2
max_value: int = 10

def generate_symmetric_matrix(size) -> np.ndarray:

    matrix: np.ndarray = np.zeros([size, size])

    # diagonal matrix

    for element in range(size):
        matrix[element, element] = np.random.randint(0, max_value)

    # fill it such that matrix is symmetric

    for element in range(size):
        filler = np.array([np.random.randint(0, max_value) for element in range(size - element - 1)])

        matrix[element, element + 1:] = filler
        matrix[element + 1:, element] = filler

    return matrix

    # verify the matrix has a real determinant

    # if np.linalg.det(matrix):
    #     return matrix
    
    # else:
    #     generate_symmetric_matrix(size)


test that function works:

In [79]:
m = generate_symmetric_matrix(2)
m

array([[5., 0.],
       [0., 9.]])

### Decomposition

use the property that a matrix A can be equated to A = PDP^T where...

#### D = diagonal matrix of eigenvalues
#### P = matrix of eigenvectors

In [80]:
eigenvalues, eigenvectors = np.linalg.eig(m)
print(eigenvalues, '\n')
print(eigenvectors)

[5. 9.] 

[[1. 0.]
 [0. 1.]]


### Find SVD of matrix

In [82]:
U, S, U_T = (np.linalg.svd(m))
print(U, '\n\n', S, '\n\n', U_T)

[[0. 1.]
 [1. 0.]] 

 [9. 5.] 

 [[0. 1.]
 [1. 0.]]


### Define our P and D matrices

In [87]:
# P: np.ndarray = 1 / abs(np.min(eigenvectors)) * eigenvectors
# P

P = eigenvectors
print(P)

[[1. 0.]
 [0. 1.]]


In [84]:
D: np.ndarray = np.zeros([size, size])
D[0, 0] = eigenvalues[0]
D[1, 1] = eigenvalues[1]
D

array([[5., 0.],
       [0., 9.]])

### More decomposition relations

We can define the relation A = (UP) * D * (PU)^T where UP is still a matrix of eigenvectors of A

Now we can normalize the columns of UP by defining another diagonal matrix Lambda that consists of singular values

In [86]:
''' compute UP '''

U_dot_P = U @ P
print(U_dot_P, '\n')

''' define Lamdba to normalize columns '''

Lambda: np.ndarray = np.zeros([size, size])
Lambda[0, 0] = 1 /np.max(abs(U_dot_P[:, 0]))
Lambda[1, 1] = 1 /np.max(abs(U_dot_P[:, 1]))

print(Lambda, '\n')

''' normalize columns '''

V: np.ndarray = U_dot_P @ Lambda
print(V)

[[0. 1.]
 [1. 0.]] 

[[1. 0.]
 [0. 1.]] 

[[0. 1.]
 [1. 0.]]


In [90]:
''' check for correctness'''

res = m @ V == V @ D
print(res)

[[ True False]
 [False  True]]
